# K-NN

refs: 
    
- PCA: https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer,KNNImputer

In [2]:
df_train = pd.read_csv("data/train.csv", index_col='PassengerId')
df_test = pd.read_csv("data/test.csv", index_col='PassengerId')
df_train.head(2)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True


# Data Treatment

In [4]:
def feature_inputer(
    df:pd.DataFrame,
    to_mode = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP'],
    to_median = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
) -> pd.DataFrame:
    for col in to_mode:
        mode_inputer = SimpleImputer(strategy='most_frequent')
        df[col] = mode_inputer.fit_transform(df[[col]])

    
    for col in to_median:
        inputer = SimpleImputer(strategy='median')
        df[col] = inputer.fit_transform(df[[col]])
    return df

def dtype_memory_reducer(df: pd.DataFrame) -> pd.DataFrame:
    df['CryoSleep'] = df['CryoSleep'].astype(bool)
    df['VIP'] = df['VIP'].astype(bool)

    df['HomePlanet'] = df['HomePlanet'].astype('category')
    df['Destination'] = df['Destination'].astype('category')
    return df

def outliers_to_log(df:pd.DataFrame) -> pd.DataFrame:
    to_log = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    for col in to_log:
        df[col] = np.log(df[col] + 1)
    return df

def cabin_inputer(df:pd.DataFrame) -> pd.DataFrame:
    cabin_features = df['Cabin'].str.split("/",expand=True)[[0,2]].rename(columns={0:'Deck',2:'side'})
    df = pd.concat([df,cabin_features],axis=1)
    df['Deck'] = df['Deck'].fillna('G').replace("T","G")
    df = df.drop(['side','Cabin'],axis=1)
    return df

def vip_knn_input(df: pd.DataFrame) -> pd.DataFrame:
    inputer = KNNImputer(n_neighbors=5)
    df['VIP'] = inputer.fit_transform(df[['VIP','RoomService']])[:,0]
    return df

def treat_dataset(df:pd.DataFrame) -> pd.DataFrame:
    df = df.drop(['Name'],axis=1)
    df = cabin_inputer(df)
    df = feature_inputer(df)
    df = vip_knn_input(df)
    df['0_bills'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) == 0
    df = outliers_to_log(df)
    df = dtype_memory_reducer(df)
    return df

def feature_enginnering(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.get_dummies(df,columns=['HomePlanet','Destination','Deck'],drop_first=True)
    return df

In [5]:
df_train = treat_dataset(df_train.copy())
df_train = feature_enginnering(df_train)

In [22]:
X_train,y_train = df_train.drop('Transported',axis=1), df_train['Transported']

In [23]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [27]:
# Make an instance of the Model
pca = PCA(.95)
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0_bills,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G
0,-0.732770,0.711945,-0.153063,-0.638181,-0.650080,-0.622995,-0.664035,-0.640034,1.174601,1.754795,-0.503664,-0.317487,0.652521,3.187347,-0.30661,-0.241218,-0.334759,-0.688215,-0.682595
1,-0.732770,-0.334037,-0.153063,1.090491,0.135040,0.646081,1.614565,0.745894,-0.851353,-0.569867,-0.503664,-0.317487,0.652521,-0.313741,-0.30661,-0.241218,-0.334759,1.453035,-0.682595
2,-0.732770,2.036857,6.533255,0.753511,2.139858,-0.622995,2.518191,0.784254,-0.851353,1.754795,-0.503664,-0.317487,0.652521,-0.313741,-0.30661,-0.241218,-0.334759,-0.688215,-0.682595
3,-0.732770,0.293552,-0.153063,-0.638181,1.790516,1.682500,2.264863,1.277886,-0.851353,1.754795,-0.503664,-0.317487,0.652521,-0.313741,-0.30661,-0.241218,-0.334759,-0.688215,-0.682595
4,-0.732770,-0.891895,-0.153063,1.464342,0.803380,1.333879,1.624920,-0.240051,-0.851353,-0.569867,-0.503664,-0.317487,0.652521,-0.313741,-0.30661,-0.241218,-0.334759,1.453035,-0.682595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,-0.732770,0.851410,6.533255,-0.638181,2.359900,-0.622995,2.009974,0.931876,-0.851353,1.754795,-0.503664,-0.317487,-1.532519,-0.313741,-0.30661,-0.241218,-0.334759,-0.688215,-0.682595
8689,1.364685,-0.752431,-0.153063,-0.638181,-0.650080,-0.622995,-0.664035,-0.640034,1.174601,-0.569867,-0.503664,3.149739,-1.532519,-0.313741,-0.30661,-0.241218,-0.334759,-0.688215,1.464998
8690,-0.732770,-0.194573,-0.153063,-0.638181,-0.650080,2.312112,-0.413730,-0.640034,-0.851353,-0.569867,-0.503664,-0.317487,0.652521,-0.313741,-0.30661,-0.241218,-0.334759,-0.688215,1.464998
8691,-0.732770,0.223820,-0.153063,-0.638181,1.721915,-0.622995,1.455450,2.302492,-0.851353,1.754795,-0.503664,-0.317487,-1.532519,-0.313741,-0.30661,-0.241218,2.987225,-0.688215,-0.682595


In [8]:
X_train = pca.fit_transform(X_train)
X_train = pd.DataFrame(X_train)
X_train

,0,1,2,3
0,9.905483,-4.480079,-1.281479,-0.051354
1,-4.417162,5.091478,2.309835,-2.128652
2,29.766714,6.442952,-2.408780,-2.764708
3,4.827569,8.456315,-1.411778,-2.592423
4,-12.345305,6.101268,4.166905,-2.847150
...,...,...,...,...
8688,12.711590,6.654042,-5.179545,-1.908982
8689,-11.055764,-3.093945,-1.166589,-0.067751
8690,-2.902550,-1.406388,3.309733,-0.808264
8691,3.725143,7.344889,-5.296079,0.701592


In [28]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, GridSearchCV

## Tuning

In [37]:
params = {
    'n_neighbors' : [22, 23, 24, 25],
    'weights' : ['uniform'],
    'metric' : ['minkowski','euclidean','manhattan']
}

In [38]:
grid = GridSearchCV(
    KNeighborsClassifier(),
    param_grid=params,
    scoring='accuracy',
    cv=KFold(5, shuffle=True, random_state=42),
    verbose=3,
    return_train_score=True
)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END metric=minkowski, n_neighbors=22, weights=uniform;, score=(train=0.791, test=0.776) total time=   0.0s
[CV 2/5] END metric=minkowski, n_neighbors=22, weights=uniform;, score=(train=0.794, test=0.760) total time=   0.0s
[CV 3/5] END metric=minkowski, n_neighbors=22, weights=uniform;, score=(train=0.794, test=0.769) total time=   0.0s
[CV 4/5] END metric=minkowski, n_neighbors=22, weights=uniform;, score=(train=0.794, test=0.778) total time=   0.0s
[CV 5/5] END metric=minkowski, n_neighbors=22, weights=uniform;, score=(train=0.794, test=0.772) total time=   0.0s
[CV 1/5] END metric=minkowski, n_neighbors=23, weights=uniform;, score=(train=0.793, test=0.779) total time=   0.0s
[CV 2/5] END metric=minkowski, n_neighbors=23, weights=uniform;, score=(train=0.797, test=0.765) total time=   0.0s
[CV 3/5] END metric=minkowski, n_neighbors=23, weights=uniform;, score=(train=0.797, test=0.770) total time=   0.0s
[CV 4/5] EN

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'metric': ['minkowski', 'euclidean', 'manhattan'],
                         'n_neighbors': [22, 23, 24, 25],
                         'weights': ['uniform']},
             return_train_score=True, scoring='accuracy', verbose=3)

In [41]:
pd.DataFrame(grid.cv_results_)[['mean_train_score','std_train_score','mean_test_score','std_test_score']].loc[grid.best_index_]

mean_train_score    0.802053
std_train_score     0.000836
mean_test_score     0.784771
std_test_score      0.007396
Name: 9, dtype: float64

In [43]:
grid.best_params_

{'metric': 'manhattan', 'n_neighbors': 23, 'weights': 'uniform'}

In [44]:
model = grid.best_estimator_
model.fit(X_train, y_train)

KNeighborsClassifier(metric='manhattan', n_neighbors=23)

## Cross Validation

# Submission

In [48]:
X_test = treat_dataset(df_test)
X_test = feature_enginnering(X_test)
X_test = pd.DataFrame(scaler.fit_transform(X_test), index=X_test.index, columns=X_test.columns)
X_test.head(2)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0_bills,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G
PassengerId,,,,,,,,,,,,,,,,,,,
0013_01,1.330443,-0.114147,-0.132689,-0.64483,-0.646513,-0.628742,-0.661615,-0.624214,1.170830,-0.553131,-0.525314,-0.315862,0.634992,-0.30408,-0.300857,-0.244898,-0.341629,-0.714311,1.490179
0018_01,-0.751630,-0.684313,-0.132689,-0.64483,0.140685,-0.628742,2.226792,-0.624214,-0.854095,-0.553131,-0.525314,-0.315862,0.634992,-0.30408,-0.300857,-0.244898,-0.341629,1.399951,-0.671060


In [49]:
y_test = model.predict(X_test)

In [50]:
submission = pd.Series(y_test, index=[X_test.index]).to_frame('Transported')
submission.to_csv('data/submission_knn_2022-11-05.csv')

In [ ]:
su